In [ ]:
import pandas as pd
import json

In [ ]:
# start coding here
top_genes = pd.concat(
    [
        pd.read_parquet(fn).iloc[:, : snakemake.params.top_n_genes]
        for fn in snakemake.input.top_genes
    ]
)

with open(snakemake.input.dataset) as f:
    eval_set = json.load(f)
eval_set[0]

In [ ]:
def row_to_conversation(idx, genes):
    return {
        "id": idx,
        "image": idx,
        "conversations": [
            {"from": "human", "value": f"{snakemake.params.question}\n<image>"},
            {
                "from": "gpt",
                "value": snakemake.params.response_prefix.format(", ".join(genes)),
            },
        ],
    }


conversations = [
    row_to_conversation(datapoint["id"], top_genes.loc[datapoint["id"]].values)
    for datapoint in eval_set
]

conversations[0]

In [ ]:
with open(snakemake.output.evaluation_dataset, "w") as f:
    json.dump(conversations, f)